In [2]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk


nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/homosapiens/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [9]:
# Create a SentimentIntensityAnalyzer object
sid_obj = SentimentIntensityAnalyzer()

# # Example text to analyze
text = "Nobel is so damn proud and dumb!"

# Get the sentiment scores
sentiment_scores = sid_obj.polarity_scores(text)

# Print the sentiment scores
print(sentiment_scores)

{'neg': 0.49, 'neu': 0.277, 'pos': 0.234, 'compound': -0.5726}


In [51]:
from googleapiclient.discovery import build
import getpass
import pandas as pd

api_key = getpass.getpass('Please enter your YouTube API key : ')


youtube = build('youtube', 'v3', developerKey=api_key)
video_id = "1DgM_CEOKVE"


# Function to get replies for a specific comment
def get_replies(youtube, parent_id, video_id):  # Added video_id as an argument
    replies = []
    next_page_token = None

    while True:
        reply_request = youtube.comments().list(
            part="snippet",
            parentId=parent_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        reply_response = reply_request.execute()

        for item in reply_response['items']:
            comment = item['snippet']
            replies.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
            })

        next_page_token = reply_response.get('nextPageToken')
        if not next_page_token:
            break

    return replies

# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,  # Directly using video_id from function parameter
                'Comment': top_comment['textDisplay'],
                'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
            })

            # Fetch replies if there are any
            if item['snippet']['totalReplyCount'] > 0:
                all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments


video_comments = get_comments_for_video(youtube, video_id)


# Create DataFrame
comments_df = pd.DataFrame(video_comments)


In [52]:
comment = comments_df["Comment"]

In [65]:
import pandas as pd  
from nltk.sentiment import SentimentIntensityAnalyzer  

# Extract the comments into a variable  
comments = comments_df["Comment"].tolist()  # Convert to a list if it's a Series  

# Create a SentimentIntensityAnalyzer object  
sid_obj = SentimentIntensityAnalyzer()  

# Function to get sentiment scores  
def get_sentiment(comment):  
    return sid_obj.polarity_scores(comment)  

# Apply the function to the comments  
sentiment_scores = [get_sentiment(comment) for comment in comments]  

# Convert sentiment scores to a DataFrame  
sentiment_df = pd.DataFrame(sentiment_scores)  

# Calculate overall mean sentiment scores  
overall_sentiment = sentiment_df.mean()  

overall_sentiment


neg         0.119568
neu         0.753562
pos         0.125315
compound   -0.019077
dtype: float64

In [68]:
# Determine the sentiment label and percentage
sentiment_label = ['Negative', 'Neutral', 'Positive']
sentiment_percentage = [
    round(overall_sentiment['neg'] * 100, 2),
    round(overall_sentiment['neu'] * 100, 2),
    round(overall_sentiment['pos'] * 100, 2)
]

# Create a new DataFrame with the sentiment label and percentage
sentiment_result = pd.DataFrame({
    'Sentiment': sentiment_label,
    'Percentage': sentiment_percentage
})

print(sentiment_result)

  Sentiment  Percentage
0  Negative       11.96
1   Neutral       75.36
2  Positive       12.53
